<h1 align=center><font size = 4>IBM/Coursera Data Science Capstone Project</font></h1>
<h1 align=center><font size = 5>Segmenting and Clustering Airbnb Listings in Zurich, Switzerland</font></h1>

## Table of Contents

* [Introduction/Business Problem](#introduction)
* [Data Collection](#data)

## Introduction/Business Problem <a name="introduction"></a>

[Airbnb](https://www.airbnb.com/) is a community where millions of hosts and travelers choose to create a free account so they can list their space and book unique accommodations anywhere in the world.

Although Airbnb listings provide enough information about the shared space, there is less information about the nearby location. For example, travelers might be interested in what kind of venues (supermarkets, restaurants, public transportation, etc) are close to the accommodation they book. In addition, travelers cannot filter Airbnb listings based on the nearby venues. In other words, each time travelers make a search for an accommodation using the Airbnb community, they may want to get direct information about the venues in the area and a list of similar Airbnb listings with same venue categories nearby.

The main objective of this project is to explore, segment and cluster Airbnb listings in Zurich, Switzerland. I will use the [Foursquare API](https://developer.foursquare.com/) to explore the areas around the Airbnb listings in Zurich. I will use the explore function to get the most common venue categories for each Airbnb listing, and then use this feature together with the prices to group the listings into clusters. I will use the k-means clustering algorithm to complete this task. Finally, I will use the Folium library to visualize the listings in Zurich and their emerging clusters.

## Data <a name="data"></a>

Based on the problem definition, I will leverage the data from the following data sources to solve the problem:

- [Airbnb Data Collection](http://tomslee.net/airbnb-data-collection-get-the-data): Here is the data provided for each Airbnb listing. Each link downloads a zip file of the data for a named city or region; in my case this is Zurich, Switzerland. The zip file holds one or more csv files. Each csv file represents a single “survey” or “scrape” of the Airbnb web site for that city. The data is collected from the public Airbnb web site and the code used is available on [GitHub](https://github.com/tomslee/airbnb-data-collection). Each csv file contains the attributes as follows:
    * room_id: A unique number identifying an Airbnb listing.
    * host_id: A unique number identifying an Airbnb host.
    * room_type: One of “Entire home/apt”, “Private room”, or “Shared room”
    * borough: A sub-region of the city or search area for which the survey is carried out. The borough is taken from a shapefile of the city that is obtained independently of the Airbnb web site. For some cities, there is no borough information; for others the borough may be a number.
    * neighborhood: As with borough: a sub-region of the city or search area for which the survey is carried out. For cities that have both, a neighborhood is smaller than a borough. For some cities there is no neighborhood information.
    * reviews: The number of reviews that a listing has received. Airbnb has said that 70% of visits end up with a review, so the number of reviews can be used to estimate the number of visits. Note that such an estimate will not be reliable for an individual listing (especially as reviews occasionally vanish from the site), but over a city as a whole it should be a useful metric of traffic.
    * overall_satisfaction: The average rating (out of five) that the listing has received from those visitors who left a review.
    * accommodates: The number of guests a listing can accommodate.
    * bedrooms: The number of bedrooms a listing offers.
    * price: The price for a night stay. In early surveys, there may be some values that were recorded by month.
    * minstay: The minimum stay for a visit, as posted by the host.
    * latitude and longitude: The latitude and longitude of the listing as posted on the Airbnb site: this may be off by a few hundred meters.
    * last_modified: the date and time that the values were read from the Airbnb web site.


- [Foursquare API](https://developer.foursquare.com/): to get the most common venues of given Airbnb listing.

Airbnb data is used to get the coordinates (latitude and longitude), neighborhood and price for each listing in Zurich, Switzerland. Having this information, I can leverage Foursquare API to explore the areas around the Airbnb listings and get the most common venue categories for each listing. Venue categories together with the price are used to segment the listings into similar clusters.

### Thank you for using this notebook!

This notebook was created by [Georgios Chatzis](https://www.linkedin.com/in/georgios-chatzis-01523293/). This notebook is part of the course on **Coursera** called *Applied Data Science Capstone*.